<a href="https://colab.research.google.com/github/dantae74/machine-learning/blob/main/stock-etc/01-crawling-naver-sise-market-sum-500.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

모두를 위한 머신러닝에서 가져왔습니다.

In [ ]:
#!pip install requests

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [7]:
def crawl(url):
  data = requests.get(url)
  # print(data)
  return data.content

In [8]:
def getSiseMarketSum(sosok, page):
  url = "https://finance.naver.com/sise/sise_market_sum.nhn?sosok={0}&page={1}".format(sosok, page)
  pageString = crawl(url)
  list = parse(pageString)
  return list

In [30]:
def parse(pageString):
  bsObj = BeautifulSoup(pageString, "html.parser")
  # print(bsObj)
  box_type_l = bsObj.find("div", {"class":"box_type_l"})
  # print(box_type_l)

  type_2 = box_type_l.find("table", {"class":"type_2"})
  # print(type_2)

  tbody = type_2.find("tbody")
  # print(tbody)

  trs = tbody.findAll("tr")
  # print(len(trs))

  tr = trs[1]
  stockInfo = getStockInfo(tr)
  print("\nstockInfo:",stockInfo)
  
  return []


In [53]:
def getStockInfo(tr):
  tds = tr.findAll("td")
  print(tds)

  # 순위
  rank = tds[0].text
  print("\ntd:", tds[0])

  # a tag => href
  aTag = tds[1].find("a")
  href = aTag["href"]
  
  # name
  name = tds[1].text

  # 현재가
  nowPrice = tds[2].text

  # 전일대비 가격

  # 시가총액
  totalPrice = tds[6].text

  # 거래량
  volume = tds[9].text

  return {"rank":rank, "href":href, "code":href[20:], 
          "name":name, "nowPrice":nowPrice, "totalPrice":totalPrice,
          "volume":volume}

In [ ]:
list = getSiseMarketSum(1, 2)

In [ ]:
# result = []

# for i in range(1, 11): # 500 = 50 * 10
#   list = getSiseMarketSum(0, i)
#   if (list is not None) :
#     print('i:', i)
#     result += list

# print(result)